In [80]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error

## Loading the data

In [81]:
df1 = pd.read_csv("DatasetTrainingFinal.csv", sep=",")

In [82]:
df1.head()

,ActualLandingTime,Airline,AircraftType,RunwayNumber,StandNumber,Date,AWNDmean,PRCPmean,TAVGmean,WDF5mean,...,IceRoad,WetDay,ShipmentWeightCat,NbPlanesLast10Mn,Hour,WeekDay,ModelAcchar,ParkingArea,TaxiTime,LogTaxiTime
0,2018-07-30 04:25:00,CAR1,B737/9-WL,2.0,4,2018-07-30 00:00:00,4.173333,0.006667,79.0,153.333333,...,0.0,1.0,M,0.0,4,0,737-900 with winglets,16223.1,4.0,1.609438
1,2018-07-30 08:14:00,CAR1,B757/2-WL,2.0,7,2018-07-30 00:00:00,4.173333,0.006667,79.0,153.333333,...,0.0,1.0,M,0.0,8,0,"757-200, -200PF with winglets",20919.9,4.0,1.609438
2,2018-07-30 08:26:00,CAR1,A321/2,1.0,8,2018-07-30 00:00:00,4.173333,0.006667,79.0,153.333333,...,0.0,1.0,M,0.0,8,0,A321-200,16337.8,3.0,1.386294
3,2018-07-30 08:42:00,CAR1,B737/9-WL,2.0,9,2018-07-30 00:00:00,4.173333,0.006667,79.0,153.333333,...,0.0,1.0,M,0.0,8,0,737-900 with winglets,16223.1,4.0,1.609438
4,2018-07-30 08:52:00,CAR1,A321/2,2.0,11,2018-07-30 00:00:00,4.173333,0.006667,79.0,153.333333,...,0.0,1.0,M,0.0,8,0,A321-200,16337.8,5.0,1.791759


In [83]:
df1.shape

(128995, 26)

In [84]:
df1.isnull().sum()

ActualLandingTime    0
Airline              0
AircraftType         0
RunwayNumber         0
StandNumber          0
Date                 0
AWNDmean             0
PRCPmean             0
TAVGmean             0
WDF5mean             0
WSF2max              0
WSF5max              0
WT01max              0
WT02max              0
WT03max              0
WT08max              0
IceRoad              0
WetDay               0
ShipmentWeightCat    0
NbPlanesLast10Mn     0
Hour                 0
WeekDay              0
ModelAcchar          0
ParkingArea          0
TaxiTime             0
LogTaxiTime          0
dtype: int64

In [90]:
len(df1['StandNumber'].unique())

160

## Clustering on StandNumber

In [109]:
variables = ['Airline','StandNumber','AircraftType','RunwayNumber','ShipmentWeightCat','NbPlanesLast10Mn', 'ParkingArea', 'TaxiTime']
SN_DB = df1[variables]

In [110]:
SN_DB.shape

(128995, 8)

One-Hot encoding

In [111]:
SN_DB['RunwayNumber'] = pd.Categorical(SN_DB['RunwayNumber'])
SN_DB['StandNumber'] = pd.Categorical(SN_DB['StandNumber'])
SN_DB['Airline'] = pd.Categorical(SN_DB['Airline'])
SN_DB['AircraftType'] = pd.Categorical(SN_DB['AircraftType'])
SN_DB['ShipmentWeightCat'] = pd.Categorical(SN_DB['ShipmentWeightCat'])

C:\Users\Cyprien\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Cyprien\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Cyprien\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://p

In [112]:
dfDummiesRunwayNumber = pd.get_dummies(df1['RunwayNumber'], prefix = 'RunwayNumber')
dfDummiesAirline = pd.get_dummies(df1['Airline'], prefix = 'Airline')
dfDummiesAircraftType = pd.get_dummies(df1['AircraftType'], prefix = 'AircraftType')
dfDummiesShipmentWeightCat = pd.get_dummies(df1['ShipmentWeightCat'], prefix = 'ShipmentWeightCat')

In [116]:
SN_DB2 = pd.concat([SN_DB, dfDummiesRunwayNumber, dfDummiesAirline, dfDummiesAircraftType,dfDummiesShipmentWeightCat], axis=1)

In [117]:
SN_DB2.drop(['RunwayNumber', 'Airline', 'AircraftType','ShipmentWeightCat'], axis=1, inplace=True)

In [119]:
SN_DB2.head()

,StandNumber,NbPlanesLast10Mn,ParkingArea,TaxiTime,RunwayNumber_0.0,RunwayNumber_1.0,RunwayNumber_2.0,RunwayNumber_3.0,RunwayNumber_4.0,RunwayNumber_5.0,...,AircraftType_CRJ/7,AircraftType_CRJ/9,AircraftType_CS/100,AircraftType_CS100,AircraftType_MD88,AircraftType_MD90,AircraftType_MD90/3,ShipmentWeightCat_L,ShipmentWeightCat_M,ShipmentWeightCat_S
0,4,0.0,16223.1,4.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,7,0.0,20919.9,4.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,8,0.0,16337.8,3.0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,9,0.0,16223.1,4.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,11,0.0,16337.8,5.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


## One Hot encoding

In [40]:
df1['RunwayNumber'] = pd.Categorical(df1['RunwayNumber'])
df1['StandNumber'] = pd.Categorical(df1['StandNumber'])
df1['Hour'] = pd.Categorical(df1['Hour'])
df1['WeekDay'] = pd.Categorical(df1['WeekDay'])
df1['Airline'] = pd.Categorical(df1['Airline'])
#df1['AircraftType'] = pd.Categorical(df1['AircraftType'])
df1['ShipmentWeightCat'] = pd.Categorical(df1['ShipmentWeightCat'])

In [41]:
dfDummiesRunwayNumber = pd.get_dummies(df1['RunwayNumber'], prefix = 'RunwayNumber')
dfDummiesStandNumber = pd.get_dummies(df1['StandNumber'], prefix = 'StandNumber')
dfDummiesHour = pd.get_dummies(df1['Hour'], prefix = 'Hour')
dfDummiesWeekDay = pd.get_dummies(df1['WeekDay'], prefix = 'WeekDay')
dfDummiesAirline = pd.get_dummies(df1['Airline'], prefix = 'Airline')
#dfDummiesModelAcchar = pd.get_dummies(df1['ModelAcchar'], prefix = 'ModelAcchar')
dfDummiesShipmentWeightCat = pd.get_dummies(df1['ShipmentWeightCat'], prefix = 'ShipmentWeightCat')

In [46]:
df2 = pd.concat([df1, dfDummiesRunwayNumber, dfDummiesStandNumber, dfDummiesHour, dfDummiesWeekDay, dfDummiesAirline, dfDummiesShipmentWeightCat], axis=1)

In [47]:
df2.shape

(128995, 230)

In [48]:
df2.head()

,Airline,RunwayNumber,StandNumber,AWNDmean,PRCPmean,TAVGmean,WDF5mean,WSF2max,WSF5max,WT01max,...,WeekDay_4,WeekDay_5,WeekDay_6,Airline_CAR1,Airline_CAR2,Airline_CAR3,Airline_CAR4,ShipmentWeightCat_L,ShipmentWeightCat_M,ShipmentWeightCat_S
0,CAR1,2.0,4,4.173333,0.006667,79.0,153.333333,15.0,19.0,1.0,...,0,0,0,1,0,0,0,0,1,0
1,CAR1,2.0,7,4.173333,0.006667,79.0,153.333333,15.0,19.0,1.0,...,0,0,0,1,0,0,0,0,1,0
2,CAR1,1.0,8,4.173333,0.006667,79.0,153.333333,15.0,19.0,1.0,...,0,0,0,1,0,0,0,0,1,0
3,CAR1,2.0,9,4.173333,0.006667,79.0,153.333333,15.0,19.0,1.0,...,0,0,0,1,0,0,0,0,1,0
4,CAR1,2.0,11,4.173333,0.006667,79.0,153.333333,15.0,19.0,1.0,...,0,0,0,1,0,0,0,0,1,0


In [51]:
X = df2.drop(['TaxiTime', 'LogTaxiTime', 'RunwayNumber', 'StandNumber', 'Hour', 'WeekDay', 'Airline', 'ShipmentWeightCat'], axis=1)
y = df2['LogTaxiTime']

In [52]:
print(X.shape)
print(y.shape)

(128995, 222)
(128995,)


## Generate training & testing sets

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False,random_state=42)

In [54]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(103196, 222) (103196,)
(25799, 222) (25799,)


## Random Forest Regressor

In [55]:
clf = RandomForestRegressor(n_estimators = 100, max_depth = 10, max_features="sqrt", n_jobs =-1)

In [56]:
clf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [57]:
y_test.head(10)

103196    2.639057
103197    2.708050
103198    2.639057
103199    1.609438
103200    2.079442
103201    1.791759
103202    1.945910
103203    2.397895
103204    2.079442
103205    1.791759
Name: LogTaxiTime, dtype: float64

In [58]:
y_pred = np.exp(clf.predict(X_test))-1

In [59]:
y_pred[:10]

array([7.70406665, 7.20348078, 7.72286009, 7.70012313, 7.74876779,
       7.08203664, 7.08203664, 7.72286009, 7.3949723 , 7.72392275])

In [60]:
np.sqrt(mean_squared_error(np.exp(y_test)-1, y_pred))

4.311796925062597

*Compute accuracy +/- 3 and 5 minutes*

In [61]:
accuracy_dataframe = pd.DataFrame(data={'y_pred':y_pred, 'y_test':np.exp(y_test)-1})

In [62]:
accuracy_dataframe['3MinutesRange'] = abs(accuracy_dataframe['y_pred'] - accuracy_dataframe['y_test']) <= 3
accuracy_dataframe['5MinutesRange'] = abs(accuracy_dataframe['y_pred'] - accuracy_dataframe['y_test']) <= 5

In [63]:
print(accuracy_dataframe['3MinutesRange'].value_counts(normalize=True) * 100)

print(accuracy_dataframe['5MinutesRange'].value_counts(normalize=True) * 100)

True     80.123261
False    19.876739
Name: 3MinutesRange, dtype: float64
True     91.86015
False     8.13985
Name: 5MinutesRange, dtype: float64


Check features importance

In [64]:
from matplotlib import pyplot as plt

In [77]:
features = X.columns
importances = clf.feature_importances_
indices = np.argsort(importances)[:100]

In [ ]:
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [78]:
list(features[indices])

['StandNumber_158',
 'StandNumber_159',
 'Hour_7',
 'StandNumber_113',
 'Hour_5',
 'StandNumber_26',
 'StandNumber_135',
 'StandNumber_146',
 'StandNumber_160',
 'StandNumber_30',
 'StandNumber_38',
 'StandNumber_104',
 'StandNumber_85',
 'Hour_6',
 'StandNumber_110',
 'StandNumber_59',
 'StandNumber_150',
 'StandNumber_63',
 'StandNumber_82',
 'StandNumber_50',
 'StandNumber_67',
 'StandNumber_17',
 'StandNumber_31',
 'StandNumber_149',
 'StandNumber_53',
 'StandNumber_4',
 'StandNumber_129',
 'StandNumber_106',
 'StandNumber_40',
 'StandNumber_111',
 'StandNumber_102',
 'StandNumber_119',
 'StandNumber_68',
 'StandNumber_23',
 'StandNumber_15',
 'StandNumber_1',
 'StandNumber_100',
 'StandNumber_120',
 'StandNumber_123',
 'StandNumber_140',
 'StandNumber_48',
 'StandNumber_44',
 'StandNumber_49',
 'StandNumber_47',
 'StandNumber_107',
 'RunwayNumber_4.0',
 'StandNumber_116',
 'StandNumber_108',
 'StandNumber_24',
 'StandNumber_145',
 'StandNumber_81',
 'StandNumber_155',
 'StandNumbe

New RF reduced

In [ ]:
cf RandomForestModel_version3

## Grid Search

A utiliser en cas de CV

In [ ]:
btscv = BlockingTimeSeriesSplit(n_splits=5)
scores = cross_val_score(model, X_train, y_train, cv=btscv, scoring=r2)

In [29]:
from pprint import pprint

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [False],
 'max_depth': [10, 35, 60, 85, 110, None],
 'max_features': ['sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 30, cv = 3, verbose=10, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 24.2min


In [ ]:
rf_random.best_params_